In [9]:
from configparser import ConfigParser
import psycopg2


"""
SELECT CURRENT_ROLE;
SELECT * FROM pg_settings WHERE setting ILIKE '%host%';
SELECT * FROM pg_settings WHERE name = 'port';
"""

def load_config(filename='database.ini', section='postgresql'):
    parser = ConfigParser()
    parser.read(filename)

    # get section, default to postgresql
    config = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            config[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return config

# https://stackoverflow.com/questions/56824788/how-to-connect-to-windows-postgres-database-from-wsl
def connect(config):
    """ Connect to the PostgreSQL database server """
    try:
        # connecting to the PostgreSQL server
        with psycopg2.connect(**config) as conn:
            print('Connected to the PostgreSQL server.')
            return conn
    except (psycopg2.DatabaseError, Exception) as error:
        print(error)


In [10]:
config = load_config()
connection = psycopg2.connect(**config)# connect(config)
print(config)

OperationalError: connection to server at "localhost" (127.0.0.1), port 5432 failed: FATAL:  database "mydata" does not exist


In [ ]:
"""
sudo systemctl start postgresql
sudo systemctl status postgresql
sudo systemctl stop postgresql

sudo -i -u postgres
psql
\password postgres


ALTER USER postgres PASSWORD 'mynewpassword'


Here's a more detailed breakdown:
1. Find the WSL IP Address:
Open your Ubuntu WSL terminal.
Run the command wsl hostname -I. This will output the IP address of your WSL instance. 
2. Configure PostgreSQL for Remote Access (if needed):
By default, PostgreSQL in WSL might only allow local connections. You may need to edit postgresql.conf and pg_hba.conf to allow connections from the WSL IP. 
Open postgresql.conf with sudo nano /etc/postgresql/<version>/main/postgresql.conf and change listen_addresses = 'localhost' to listen_addresses = '*' to allow connections from any IP. 
Open pg_hba.conf with sudo nano /etc/postgresql/<version>/main/pg_hba.conf and add a line like host all all <WSL_IP>/32 md5 to allow connections from your WSL IP (replace <WSL_IP> with the actual IP). 
Restart the PostgreSQL service after making these changes. 
3. Connect from Windows:
Use the WSL IP address you found earlier and the default PostgreSQL port (5432) in your connection string or configuration for your chosen PostgreSQL client tool (e.g., pgAdmin). 
Example:
If wsl hostname -I returns 172.20.37.128, and you're using pgAdmin, you would: 
Open pgAdmin.
Right-click on "Servers" and select "Register" -> "Server".
Name the server connection (e.g., "WSL PostgreSQL").
In the "Connection" tab, enter:
Host name/address: 172.20.37.128
Port: 5432
Username: The PostgreSQL username you want to use.
Password: The password for that user.
Click "Save".
"""

'\nHere\'s a more detailed breakdown:\n1. Find the WSL IP Address:\nOpen your Ubuntu WSL terminal.\nRun the command wsl hostname -I. This will output the IP address of your WSL instance. \n2. Configure PostgreSQL for Remote Access (if needed):\nBy default, PostgreSQL in WSL might only allow local connections. You may need to edit postgresql.conf and pg_hba.conf to allow connections from the WSL IP. \nOpen postgresql.conf with sudo nano /etc/postgresql/<version>/main/postgresql.conf and change listen_addresses = \'localhost\' to listen_addresses = \'*\' to allow connections from any IP. \nOpen pg_hba.conf with sudo nano /etc/postgresql/<version>/main/pg_hba.conf and add a line like host all all <WSL_IP>/32 md5 to allow connections from your WSL IP (replace <WSL_IP> with the actual IP). \nRestart the PostgreSQL service after making these changes. \n3. Connect from Windows:\nUse the WSL IP address you found earlier and the default PostgreSQL port (5432) in your connection string or config